C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64>python -m pip install google-cloud-bigquery

In [1]:
import pyodbc 
from datetime import datetime, date, timedelta
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
src_conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=Localhost;'
                      'Database=CollectionStrategy;'
                      'Trusted_Connection=yes;')

dst_conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=Localhost;'
                      'Database=DW;'
                      'Trusted_Connection=yes;')

In [3]:
rowlog = 0
rowbatch = 10000
src_qry = 'SELECT * FROM dbo.TREATMENT1 order by loan_id'
dst_tbl = "dbo.tm"
dst_ins = "insert into {} values(%s); ".format(dst_tbl)

type_str = type('str')
type_datetime = type(datetime.now())
type_int = type(1)
type_float = type(1.0)
type_None = type(None)

#todo: handle blob data type
 
def convert2str(record):
    res = []
    for item in record:
        if type(item)==type_None:
            res.append('NULL')
        elif type(item)==type_str:
            res.append("'"+item+"'")
        elif type(item)==type_datetime:
            res.append("'"+str(item)+"'")
        else:  # for numeric values
            res.append(str(item))
    return ','.join(res)
                                                     
src_cur = src_conn.cursor()
dst_cur = dst_conn.cursor()
#enddate = date.today()

src_cur.execute(src_qry)

rowinsert = 0
sql = ""
for row in src_cur:
    str_row = convert2str(row)        
    #print(str_row)
    sql += dst_ins%(str_row)
    rowinsert += 1
    
    if rowinsert == rowbatch:
        #print(sql)
        dst_cur.execute(sql)
        print("{} rows inserted".format(rowinsert))
        sql = ""
        rowinsert = 0
        
if rowinsert != rowbatch:
    dst_cur.execute(sql)
    print("{} rows inserted".format(rowinsert))
    
dst_cur.commit()
print('finished')

dst_cur.close()
src_cur.close()


10000 rows inserted
10000 rows inserted
4990 rows inserted
finished
